In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [12]:
community = 'north_birmingham'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_tiered.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2704.4
Basic Enclosure                                                     4882.8
Enhanced Enclosure                                                  7008.8
Mininum Efficiency Heat Pump with Existing Heat Backup             13743.1
Mininum Efficiency Heat Pump with Electric Heat Backup             13857.3
Mininum Efficiency Whole Home Electrification                      14860.9
High Efficiency Heat Pump with Electric Heat Backup                24368.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28330.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29835.7
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Electric Clothes Dryer                                              15468.784195
Electric Cooking                                                   105517.793990
Induction Cooking                                                  307343.860182
Heat Pump Clothes Dryer                                            261609.725077
Heat Pump Water Heater                                             306595.431672
Mininum Efficiency Heat Pump with Existing Heat Backup             307321.780515
Mininum Efficiency Heat Pump with Electric Heat Backup             308658.385482
Basic Enclosure                                                    304262.362179
Mininum Efficiency Whole Home Electrification                      195582.227004
Enhanced Enclosure                                                 305785.048275
High Efficiency Heat Pump with Electric Heat Backup                308658.385482
High Efficiency Whole Home Electrification                         309000.000000
Basic Enclosure

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1124,309000.000000,59.007244,49.638911,68.540000
Basic Enclosure + High Efficiency Whole Home Electrification,1124,309000.000000,58.236032,49.242550,67.589866
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1124,309000.000000,57.455489,47.904741,67.146831
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1124,309000.000000,56.684274,46.986339,66.433945
High Efficiency Whole Home Electrification,1124,309000.000000,52.907504,44.670000,61.536383
High Efficiency Heat Pump with Electric Heat Backup,1120,308658.385482,41.199938,31.409198,52.032637
Mininum Efficiency Whole Home Electrification,605,195582.227004,35.888436,26.418048,47.766781
Enhanced Enclosure,1114,305785.048275,24.081021,12.910050,33.894986
Mininum Efficiency Heat Pump with Electric Heat Backup,1120,308658.385482,24.045435,12.520000,34.411597


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, gas fuel type, all income  

Segment 2. Occupied, Single family detached, pre 1940, electric fuel type, all income  

Segment 3. Occupied, Single family detached, 1940-1979, gas fuel type, all income  

Segment 4. Occupied, Single family detached, 1940-1979, electric fuel type, all income 


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_heating_fuel = ['Natural Gas']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_heating_fuel = ['Electricity']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_heating_fuel = ['Natural Gas']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Detached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_heating_fuel = ['Electricity']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg3_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg4_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

In [20]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,30,15121.276596,68.827065,62.480000,75.485000
Basic Enclosure + High Efficiency Whole Home Electrification,30,15121.276596,68.391486,62.172500,75.095000
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,30,15121.276596,67.018841,60.150000,73.530000
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,30,15121.276596,66.583261,59.880000,73.027500
High Efficiency Whole Home Electrification,30,15121.276596,63.276341,57.670000,70.690000
High Efficiency Heat Pump with Electric Heat Backup,30,15121.276596,51.317754,42.147500,61.610000
Mininum Efficiency Whole Home Electrification,30,15121.276596,48.389746,41.385000,55.990000
Mininum Efficiency Heat Pump with Electric Heat Backup,30,15121.276596,38.530471,28.180000,47.620000
Mininum Efficiency Heat Pump with Existing Heat Backup,30,15121.276596,32.746775,20.100000,47.350000


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,21,10354.787234,58.274286,50.340625,65.115000
Basic Enclosure + High Efficiency Whole Home Electrification,21,10354.787234,57.561429,50.289375,64.060000
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,21,10354.787234,57.017619,48.660000,64.608333
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,21,10354.787234,56.304127,48.517500,63.255000
High Efficiency Whole Home Electrification,21,10354.787234,49.683333,44.599375,55.186250
High Efficiency Heat Pump with Electric Heat Backup,21,10354.787234,42.264603,32.285625,51.410000
Enhanced Enclosure,21,10354.787234,30.593810,25.022500,38.887500
Basic Enclosure,21,10354.787234,28.111111,22.002500,36.339375
Mininum Efficiency Heat Pump with Electric Heat Backup,21,10354.787234,23.159206,12.776250,32.930000


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,185,57871.755319,67.630457,62.526464,73.500000
Basic Enclosure + High Efficiency Whole Home Electrification,185,57871.755319,66.830111,61.824442,72.279317
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,185,57871.755319,65.779781,60.104987,71.689260
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,185,57871.755319,64.979289,59.591625,70.936116
High Efficiency Whole Home Electrification,185,57871.755319,61.158866,56.555232,66.738155
High Efficiency Heat Pump with Electric Heat Backup,185,57871.755319,49.056280,41.905321,57.694648
Mininum Efficiency Whole Home Electrification,185,57871.755319,45.158907,40.963571,50.734905
Mininum Efficiency Heat Pump with Electric Heat Backup,185,57871.755319,34.590905,28.178386,42.992871
Enhanced Enclosure,185,57871.755319,32.768593,26.780000,38.554701


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,98,31752.688588,57.154134,51.411321,62.400000
Basic Enclosure + High Efficiency Whole Home Electrification,98,31752.688588,56.429900,50.242000,62.129750
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,98,31752.688588,55.530300,48.716889,60.578538
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,98,31752.688588,54.804835,48.045822,60.099924
High Efficiency Whole Home Electrification,98,31752.688588,50.026812,44.212000,57.402455
High Efficiency Heat Pump with Electric Heat Backup,98,31752.688588,41.586031,34.546141,47.962958
Enhanced Enclosure,98,31752.688588,25.726302,17.568773,33.388750
Mininum Efficiency Heat Pump with Electric Heat Backup,98,31752.688588,23.450598,17.067200,31.634964
Mininum Efficiency Heat Pump with Existing Heat Backup,98,31752.688588,23.450598,17.067200,31.634964


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2679.9
Basic Enclosure                                                     6830.6
Enhanced Enclosure                                                  9962.4
Mininum Efficiency Heat Pump with Existing Heat Backup             22228.2
Mininum Efficiency Heat Pump with Electric Heat Backup             22250.5
Mininum Efficiency Whole Home Electrification                      25026.2
High Efficiency Heat Pump with Electric Heat Backup                26207.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28869.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30729.5
Basic Enclos

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1884.2
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2761.4
Basic Enclosure                                                     7443.2
Enhanced Enclosure                                                  9869.5
Mininum Efficiency Heat Pump with Electric Heat Backup             16978.8
Mininum Efficiency Heat Pump with Existing Heat Backup             16978.8
High Efficiency Heat Pump with Electric Heat Backup                27240.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31274.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32889.1
High Efficiency Whole Home Electrification                         33577.5
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2752.1
Basic Enclosure                                                     6506.5
Enhanced Enclosure                                                  9205.6
Mininum Efficiency Heat Pump with Existing Heat Backup             14552.9
Mininum Efficiency Heat Pump with Electric Heat Backup             14812.6
Mininum Efficiency Whole Home Electrification                      17636.7
High Efficiency Heat Pump with Electric Heat Backup                26358.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31042.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32900.8
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1813.6
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2690.6
Basic Enclosure                                                     6006.3
Enhanced Enclosure                                                  7989.9
Mininum Efficiency Heat Pump with Electric Heat Backup             17261.7
Mininum Efficiency Heat Pump with Existing Heat Backup             17261.7
High Efficiency Heat Pump with Electric Heat Backup                26197.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30362.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31658.1
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.09699
Enhanced Enclosure                                                 0.07732
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05351
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05078
Basic Enclosure + High Efficiency Whole Home Electrification       0.04831
Heat Pump Water Heater                                             0.04673
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04617
High Efficiency Heat Pump with Electric Heat Backup                0.04345
High Efficiency Whole Home Electrification                         0.04206
Mininum Efficiency Whole Home Electrification                      0.03189
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02842
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02501
Heat Pump Clothes Dryer                                            0.01202
Electric Coo

segment2


upgrade_name
Basic Enclosure                                                    0.13372
Enhanced Enclosure                                                 0.10872
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.06091
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05868
Basic Enclosure + High Efficiency Whole Home Electrification       0.05522
High Efficiency Heat Pump with Electric Heat Backup                0.05385
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.05347
High Efficiency Whole Home Electrification                         0.04944
Mininum Efficiency Heat Pump with Electric Heat Backup             0.04783
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04783
Heat Pump Water Heater                                             0.04466
Mininum Efficiency Whole Home Electrification                      0.01252
Heat Pump Clothes Dryer                                            0.01195
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.10722
Enhanced Enclosure                                                 0.08830
Heat Pump Water Heater                                             0.05143
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04940
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04759
Basic Enclosure + High Efficiency Whole Home Electrification       0.04512
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04372
High Efficiency Heat Pump with Electric Heat Backup                0.04199
Mininum Efficiency Whole Home Electrification                      0.04151
High Efficiency Whole Home Electrification                         0.04096
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03643
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02494
Heat Pump Clothes Dryer                                            0.01318
Electric Coo

segment4


upgrade_name
Basic Enclosure                                                    0.11572
Enhanced Enclosure                                                 0.09727
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05249
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05098
High Efficiency Heat Pump with Electric Heat Backup                0.04778
Basic Enclosure + High Efficiency Whole Home Electrification       0.04753
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04636
High Efficiency Whole Home Electrification                         0.04392
Mininum Efficiency Heat Pump with Electric Heat Backup             0.04243
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04243
Heat Pump Water Heater                                             0.04124
Mininum Efficiency Whole Home Electrification                      0.03868
Heat Pump Clothes Dryer                                            0.01368
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00458
Enhanced Enclosure                                                 0.00366
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00262
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00248
Basic Enclosure + High Efficiency Whole Home Electrification       0.00236
High Efficiency Heat Pump with Electric Heat Backup                0.00230
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00225
Mininum Efficiency Whole Home Electrification                      0.00223
Heat Pump Water Heater                                             0.00220
High Efficiency Whole Home Electrification                         0.00219
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00209
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00181
Electric Cooking                                                   0.00146
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00367
Enhanced Enclosure                                                 0.00298
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00168
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00162
Basic Enclosure + High Efficiency Whole Home Electrification       0.00152
High Efficiency Heat Pump with Electric Heat Backup                0.00148
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00148
Mininum Efficiency Whole Home Electrification                      0.00139
High Efficiency Whole Home Electrification                         0.00137
Electric Cooking                                                   0.00132
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00131
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00131
Heat Pump Water Heater                                             0.00130
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00477
Enhanced Enclosure                                                 0.00393
Mininum Efficiency Whole Home Electrification                      0.00281
Heat Pump Water Heater                                             0.00261
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00261
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00228
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00218
Basic Enclosure + High Efficiency Whole Home Electrification       0.00208
High Efficiency Heat Pump with Electric Heat Backup                0.00206
High Efficiency Whole Home Electrification                         0.00202
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00200
Electric Cooking                                                   0.00159
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00147
Induction Co

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00353
Basic Enclosure                                                    0.00317
Enhanced Enclosure                                                 0.00267
Electric Cooking                                                   0.00148
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00147
Heat Pump Water Heater                                             0.00147
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00143
Basic Enclosure + High Efficiency Whole Home Electrification       0.00134
High Efficiency Heat Pump with Electric Heat Backup                0.00131
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00130
High Efficiency Whole Home Electrification                         0.00124
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00116
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00116
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.30134
Enhanced Enclosure                                                 0.24255
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.16990
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.16122
Basic Enclosure + High Efficiency Whole Home Electrification       0.15310
Heat Pump Water Heater                                             0.14776
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.14630
High Efficiency Heat Pump with Electric Heat Backup                0.14324
High Efficiency Whole Home Electrification                         0.13798
Mininum Efficiency Whole Home Electrification                      0.12649
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11455
Mininum Efficiency Heat Pump with Existing Heat Backup             0.10161
Electric Cooking                                                   0.08381
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.34302
Enhanced Enclosure                                                 0.28110
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15565
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15030
High Efficiency Heat Pump with Electric Heat Backup                0.14161
Basic Enclosure + High Efficiency Whole Home Electrification       0.14089
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13674
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13413
Mininum Efficiency Heat Pump with Existing Heat Backup             0.13413
High Efficiency Whole Home Electrification                         0.12771
Heat Pump Water Heater                                             0.09194
Electric Cooking                                                   0.08105
Mininum Efficiency Whole Home Electrification                      0.08051
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.31310
Enhanced Enclosure                                                 0.26100
Heat Pump Water Heater                                             0.17243
Mininum Efficiency Whole Home Electrification                      0.15546
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14925
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14362
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13708
Basic Enclosure + High Efficiency Whole Home Electrification       0.13628
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13189
High Efficiency Heat Pump with Electric Heat Backup                0.12907
High Efficiency Whole Home Electrification                         0.12793
Electric Cooking                                                   0.09360
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08721
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.30262
Mininum Efficiency Whole Home Electrification                      0.25558
Enhanced Enclosure                                                 0.25527
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13734
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13350
High Efficiency Heat Pump with Electric Heat Backup                0.12662
Basic Enclosure + High Efficiency Whole Home Electrification       0.12428
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12133
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11896
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11896
High Efficiency Whole Home Electrification                         0.11596
Heat Pump Water Heater                                             0.10872
Electric Cooking                                                   0.09743
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    24168164.6
Basic Enclosure + High Efficiency Whole Home Electrification       23931069.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23594798.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        23357703.4
High Efficiency Whole Home Electrification                         20844899.8
High Efficiency Heat Pump with Electric Heat Backup                17219305.6
Mininum Efficiency Whole Home Electrification                      12069132.5
Enhanced Enclosure                                                 11647196.8
Basic Enclosure                                                    10018315.5
Mininum Efficiency Heat Pump with Electric Heat Backup              9560695.3
Mininum Efficiency Heat Pump with Existing Heat Backup              8406530.1
Heat Pump Water Heater                                              1893507.5
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    20284435.8
Basic Enclosure + High Efficiency Whole Home Electrification       20026164.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     19984033.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        19725762.4
High Efficiency Whole Home Electrification                         17189065.0
High Efficiency Heat Pump with Electric Heat Backup                15188631.0
Enhanced Enclosure                                                 11110894.2
Basic Enclosure                                                    10305829.6
Mininum Efficiency Heat Pump with Electric Heat Backup              8408329.0
Mininum Efficiency Heat Pump with Existing Heat Backup              8408269.1
Heat Pump Water Heater                                              1196059.8
Heat Pump Clothes Dryer                                              256132.1
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    92945998.1
Basic Enclosure + High Efficiency Whole Home Electrification       91069824.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     90620465.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        88744291.4
High Efficiency Whole Home Electrification                         78041252.5
High Efficiency Heat Pump with Electric Heat Backup                64045519.0
Enhanced Enclosure                                                 47040659.6
Mininum Efficiency Whole Home Electrification                      42363461.4
Basic Enclosure                                                    40374777.2
Mininum Efficiency Heat Pump with Electric Heat Backup             31227785.1
Mininum Efficiency Heat Pump with Existing Heat Backup             21008590.1
Heat Pump Water Heater                                              8145066.7
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    52287276.6
Basic Enclosure + High Efficiency Whole Home Electrification       51652025.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51242037.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50606786.3
High Efficiency Whole Home Electrification                         45676525.3
High Efficiency Heat Pump with Electric Heat Backup                39741905.8
Enhanced Enclosure                                                 24676789.5
Mininum Efficiency Heat Pump with Electric Heat Backup             23256109.1
Mininum Efficiency Heat Pump with Existing Heat Backup             23256033.2
Basic Enclosure                                                    22069000.1
Heat Pump Water Heater                                              3523516.2
Heat Pump Clothes Dryer                                              948020.5
Mininum Efficiency Whole Home Electrification      

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1177261.0
Basic Enclosure + High Efficiency Whole Home Electrification       1170756.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1151729.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1145224.8
High Efficiency Whole Home Electrification                         1086096.1
High Efficiency Heat Pump with Electric Heat Backup                 912898.5
Mininum Efficiency Whole Home Electrification                       845356.4
Mininum Efficiency Heat Pump with Electric Heat Backup              702804.4
Mininum Efficiency Heat Pump with Existing Heat Backup              608256.6
Enhanced Enclosure                                                  551398.4
Basic Enclosure                                                     473429.0
Heat Pump Water Heater                                               89345.1
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    560029.8
Basic Enclosure + High Efficiency Whole Home Electrification       552944.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     550295.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        543210.2
High Efficiency Whole Home Electrification                         475116.6
High Efficiency Heat Pump with Electric Heat Backup                416659.9
Enhanced Enclosure                                                 304798.6
Basic Enclosure                                                    282712.8
Mininum Efficiency Heat Pump with Electric Heat Backup             230660.4
Mininum Efficiency Heat Pump with Existing Heat Backup             230658.8
Heat Pump Water Heater                                              34897.3
Heat Pump Clothes Dryer                                              7026.3
Mininum Efficiency Whole Home Electrification                        4751.0

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4256234.2
Basic Enclosure + High Efficiency Whole Home Electrification       4204765.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4150794.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4099325.5
High Efficiency Whole Home Electrification                         3847362.4
High Efficiency Heat Pump with Electric Heat Backup                3137430.3
Mininum Efficiency Whole Home Electrification                      2868637.0
Mininum Efficiency Heat Pump with Electric Heat Backup             2237221.1
Enhanced Enclosure                                                 2093927.8
Basic Enclosure                                                    1796848.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1235328.3
Heat Pump Water Heater                                              414039.3
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1470897.5
Basic Enclosure + High Efficiency Whole Home Electrification       1453471.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1434356.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1416930.4
High Efficiency Whole Home Electrification                         1289549.5
High Efficiency Heat Pump with Electric Heat Backup                1090023.7
Enhanced Enclosure                                                  676986.0
Mininum Efficiency Heat Pump with Electric Heat Backup              637780.8
Mininum Efficiency Heat Pump with Existing Heat Backup              637778.7
Basic Enclosure                                                     605432.6
Heat Pump Water Heater                                              125327.9
Mininum Efficiency Whole Home Electrification                        52262.7
Heat Pump Clothes Dryer                                        

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    76587755.6
Basic Enclosure + High Efficiency Whole Home Electrification       75845181.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74914264.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        74171690.4
High Efficiency Whole Home Electrification                         68392124.5
High Efficiency Heat Pump with Electric Heat Backup                56766490.2
Mininum Efficiency Whole Home Electrification                      47867485.1
Mininum Efficiency Heat Pump with Electric Heat Backup             38541896.9
Enhanced Enclosure                                                 36538834.1
Mininum Efficiency Heat Pump with Existing Heat Backup             34154251.2
Basic Enclosure                                                    31124606.0
Heat Pump Water Heater                                              5987797.3
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    51877789.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51185662.0
Basic Enclosure + High Efficiency Whole Home Electrification       51099836.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50407708.7
High Efficiency Whole Home Electrification                         44401490.8
High Efficiency Heat Pump with Electric Heat Backup                39943234.0
Enhanced Enclosure                                                 28727164.8
Basic Enclosure                                                    26437403.3
Mininum Efficiency Heat Pump with Electric Heat Backup             23582445.9
Mininum Efficiency Heat Pump with Existing Heat Backup             23582322.9
Heat Pump Water Heater                                              2462022.8
Heat Pump Clothes Dryer                                              503774.1
Mininum Efficiency Whole Home Electrification      

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    280365107.1
Basic Enclosure + High Efficiency Whole Home Electrification       275045104.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     273453930.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        268133927.7
High Efficiency Whole Home Electrification                         243769224.1
High Efficiency Heat Pump with Electric Heat Backup                196874460.1
Mininum Efficiency Whole Home Electrification                      158668716.2
Enhanced Enclosure                                                 139044825.6
Basic Enclosure                                                    117897327.3
Mininum Efficiency Heat Pump with Electric Heat Backup             117507249.1
Mininum Efficiency Heat Pump with Existing Heat Backup              73450763.9
Heat Pump Water Heater                                              27306522.5
Heat Pump Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    136847817.4
Basic Enclosure + High Efficiency Whole Home Electrification       135061772.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     134198375.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        132412330.9
High Efficiency Whole Home Electrification                         120595376.4
High Efficiency Heat Pump with Electric Heat Backup                105326627.7
Mininum Efficiency Heat Pump with Electric Heat Backup              65200101.9
Mininum Efficiency Heat Pump with Existing Heat Backup              65199961.1
Enhanced Enclosure                                                  64762944.2
Basic Enclosure                                                     57713497.0
Heat Pump Water Heater                                               9288693.3
Mininum Efficiency Whole Home Electrification                        3788162.2
Heat Pump Clothes Dryer                

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1598.3
Basic Enclosure + High Efficiency Whole Home Electrification       1582.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1560.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1544.7
High Efficiency Whole Home Electrification                         1378.5
High Efficiency Heat Pump with Electric Heat Backup                1138.7
Mininum Efficiency Whole Home Electrification                       798.2
Enhanced Enclosure                                                  770.3
Basic Enclosure                                                     662.5
Mininum Efficiency Heat Pump with Electric Heat Backup              632.3
Mininum Efficiency Heat Pump with Existing Heat Backup              555.9
Heat Pump Water Heater                                              125.2
Heat Pump Clothes Dryer                                              26.6
Induction Cooking        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1958.9
Basic Enclosure + High Efficiency Whole Home Electrification       1934.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1929.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1905.0
High Efficiency Whole Home Electrification                         1660.0
High Efficiency Heat Pump with Electric Heat Backup                1466.8
Enhanced Enclosure                                                 1073.0
Basic Enclosure                                                     995.3
Mininum Efficiency Heat Pump with Electric Heat Backup              812.0
Mininum Efficiency Heat Pump with Existing Heat Backup              812.0
Heat Pump Water Heater                                              123.3
Heat Pump Clothes Dryer                                              26.4
Mininum Efficiency Whole Home Electrification                        23.6
Induction Cooking        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1606.1
Basic Enclosure + High Efficiency Whole Home Electrification       1573.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1565.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1533.5
High Efficiency Whole Home Electrification                         1348.5
High Efficiency Heat Pump with Electric Heat Backup                1106.7
Enhanced Enclosure                                                  812.8
Mininum Efficiency Whole Home Electrification                       732.0
Basic Enclosure                                                     697.7
Mininum Efficiency Heat Pump with Electric Heat Backup              539.6
Mininum Efficiency Heat Pump with Existing Heat Backup              363.0
Heat Pump Water Heater                                              141.5
Heat Pump Clothes Dryer                                              29.1
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1646.7
Basic Enclosure + High Efficiency Whole Home Electrification       1626.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1613.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1593.8
High Efficiency Whole Home Electrification                         1438.5
High Efficiency Heat Pump with Electric Heat Backup                1251.6
Enhanced Enclosure                                                  777.2
Mininum Efficiency Heat Pump with Electric Heat Backup              732.4
Mininum Efficiency Heat Pump with Existing Heat Backup              732.4
Basic Enclosure                                                     695.0
Heat Pump Water Heater                                              111.0
Mininum Efficiency Whole Home Electrification                        70.1
Heat Pump Clothes Dryer                                              30.2
Induction Cooking        

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    77.9
Basic Enclosure + High Efficiency Whole Home Electrification       77.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     76.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        75.7
High Efficiency Whole Home Electrification                         71.8
High Efficiency Heat Pump with Electric Heat Backup                60.4
Mininum Efficiency Whole Home Electrification                      55.9
Mininum Efficiency Heat Pump with Electric Heat Backup             46.5
Mininum Efficiency Heat Pump with Existing Heat Backup             40.2
Enhanced Enclosure                                                 36.5
Basic Enclosure                                                    31.3
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.1
Basic Enclosure + High Efficiency Whole Home Electrification       53.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.5
High Efficiency Whole Home Electrification                         45.9
High Efficiency Heat Pump with Electric Heat Backup                40.2
Enhanced Enclosure                                                 29.4
Basic Enclosure                                                    27.3
Mininum Efficiency Heat Pump with Electric Heat Backup             22.3
Mininum Efficiency Heat Pump with Existing Heat Backup             22.3
Heat Pump Water Heater                                              3.6
Mininum Efficiency Whole Home Electrification                       2.6
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73.5
Basic Enclosure + High Efficiency Whole Home Electrification       72.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70.8
High Efficiency Whole Home Electrification                         66.5
High Efficiency Heat Pump with Electric Heat Backup                54.2
Mininum Efficiency Whole Home Electrification                      49.6
Mininum Efficiency Heat Pump with Electric Heat Backup             38.7
Enhanced Enclosure                                                 36.2
Basic Enclosure                                                    31.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.3
Heat Pump Water Heater                                              7.2
Electric Cooking                                                    1.6
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    46.3
Basic Enclosure + High Efficiency Whole Home Electrification       45.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     45.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.6
High Efficiency Whole Home Electrification                         40.6
High Efficiency Heat Pump with Electric Heat Backup                34.3
Enhanced Enclosure                                                 21.3
Mininum Efficiency Heat Pump with Electric Heat Backup             20.1
Mininum Efficiency Heat Pump with Existing Heat Backup             20.1
Basic Enclosure                                                    19.1
Mininum Efficiency Whole Home Electrification                       6.4
Heat Pump Water Heater                                              3.9
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    68.8
Basic Enclosure + High Efficiency Whole Home Electrification       68.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        66.6
High Efficiency Whole Home Electrification                         63.3
High Efficiency Heat Pump with Electric Heat Backup                51.3
Mininum Efficiency Whole Home Electrification                      48.4
Mininum Efficiency Heat Pump with Electric Heat Backup             38.5
Mininum Efficiency Heat Pump with Existing Heat Backup             32.7
Enhanced Enclosure                                                 32.2
Basic Enclosure                                                    27.5
Heat Pump Water Heater                                              6.4
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    58.3
Basic Enclosure + High Efficiency Whole Home Electrification       57.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     57.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        56.3
High Efficiency Whole Home Electrification                         49.7
High Efficiency Heat Pump with Electric Heat Backup                42.3
Enhanced Enclosure                                                 30.6
Basic Enclosure                                                    28.1
Mininum Efficiency Heat Pump with Electric Heat Backup             23.2
Mininum Efficiency Heat Pump with Existing Heat Backup             23.2
Heat Pump Water Heater                                              4.7
Mininum Efficiency Whole Home Electrification                       2.8
Electric Cooking                                                    2.0
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.6
Basic Enclosure + High Efficiency Whole Home Electrification       66.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     65.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.0
High Efficiency Whole Home Electrification                         61.2
High Efficiency Heat Pump with Electric Heat Backup                49.1
Mininum Efficiency Whole Home Electrification                      45.2
Mininum Efficiency Heat Pump with Electric Heat Backup             34.6
Enhanced Enclosure                                                 32.8
Basic Enclosure                                                    28.0
Mininum Efficiency Heat Pump with Existing Heat Backup             19.1
Heat Pump Water Heater                                              7.2
Electric Cooking                                                    1.6
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    57.2
Basic Enclosure + High Efficiency Whole Home Electrification       56.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     55.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54.8
High Efficiency Whole Home Electrification                         50.0
High Efficiency Heat Pump with Electric Heat Backup                41.6
Enhanced Enclosure                                                 25.7
Mininum Efficiency Heat Pump with Electric Heat Backup             23.5
Mininum Efficiency Heat Pump with Existing Heat Backup             23.5
Basic Enclosure                                                    22.8
Mininum Efficiency Whole Home Electrification                       7.2
Heat Pump Water Heater                                              5.2
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5064.9
Basic Enclosure + High Efficiency Whole Home Electrification       5015.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4954.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4905.1
High Efficiency Whole Home Electrification                         4522.9
High Efficiency Heat Pump with Electric Heat Backup                3754.1
Mininum Efficiency Whole Home Electrification                      3165.6
Mininum Efficiency Heat Pump with Electric Heat Backup             2548.9
Enhanced Enclosure                                                 2416.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2258.7
Basic Enclosure                                                    2058.3
Heat Pump Water Heater                                              396.0
Electric Cooking                                                     81.9
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5010.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4943.2
Basic Enclosure + High Efficiency Whole Home Electrification       4934.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4868.1
High Efficiency Whole Home Electrification                         4288.0
High Efficiency Heat Pump with Electric Heat Backup                3857.5
Enhanced Enclosure                                                 2774.3
Basic Enclosure                                                    2553.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2277.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2277.4
Heat Pump Water Heater                                              253.9
Mininum Efficiency Whole Home Electrification                       151.7
Electric Cooking                                                     79.2
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4844.6
Basic Enclosure + High Efficiency Whole Home Electrification       4752.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4725.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4633.2
High Efficiency Whole Home Electrification                         4212.2
High Efficiency Heat Pump with Electric Heat Backup                3401.9
Mininum Efficiency Whole Home Electrification                      2741.7
Enhanced Enclosure                                                 2402.6
Basic Enclosure                                                    2037.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2030.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1269.2
Heat Pump Water Heater                                              474.5
Electric Cooking                                                     91.5
Heat Pump Clothes Dryer  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4309.8
Basic Enclosure + High Efficiency Whole Home Electrification       4253.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4226.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4170.1
High Efficiency Whole Home Electrification                         3798.0
High Efficiency Heat Pump with Electric Heat Backup                3317.1
Mininum Efficiency Heat Pump with Electric Heat Backup             2053.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2053.4
Enhanced Enclosure                                                 2039.6
Basic Enclosure                                                    1817.6
Mininum Efficiency Whole Home Electrification                       463.5
Heat Pump Water Heater                                              292.5
Electric Cooking                                                     95.3
Heat Pump Clothes Dryer  

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [34]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [35]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [36]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,18.1,7.8
Basic Enclosure + High Efficiency Whole Home Electrification,18.1,8.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,8.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,8.2
High Efficiency Whole Home Electrification,18.1,9.3
High Efficiency Heat Pump with Electric Heat Backup,18.1,10.9
Enhanced Enclosure,18.1,13.0
Mininum Efficiency Whole Home Electrification,18.1,13.1
Basic Enclosure,18.1,13.9


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,18.0,7.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18.0,7.9
Basic Enclosure + High Efficiency Whole Home Electrification,18.0,7.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18.0,8.1
High Efficiency Whole Home Electrification,18.0,9.6
High Efficiency Heat Pump with Electric Heat Backup,18.0,10.7
Enhanced Enclosure,18.0,12.5
Basic Enclosure,18.0,13.0
Mininum Efficiency Heat Pump with Electric Heat Backup,18.0,14.6


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,13.0,5.6
Basic Enclosure + High Efficiency Whole Home Electrification,13.0,5.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,13.0,5.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,13.0,5.9
High Efficiency Whole Home Electrification,13.0,6.8
High Efficiency Heat Pump with Electric Heat Backup,13.0,7.9
Enhanced Enclosure,13.0,9.2
Mininum Efficiency Whole Home Electrification,13.0,9.7
Basic Enclosure,13.0,9.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.0,6.3
Basic Enclosure + High Efficiency Whole Home Electrification,15.0,6.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.0,6.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.0,6.6
High Efficiency Whole Home Electrification,15.0,7.3
High Efficiency Heat Pump with Electric Heat Backup,15.0,8.3
Mininum Efficiency Heat Pump with Existing Heat Backup,15.0,10.9
Mininum Efficiency Heat Pump with Electric Heat Backup,15.0,10.9
Enhanced Enclosure,15.0,11.1


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, gas fuel type, 0-80% AMI  

Segment 2. Occupied, Single family detached, pre 1940, electric fuel type, 0-80% AMI    

Segment 3. Occupied, Single family detached, 1940-1979, gas fuel type, 0-80% AMI  

Segment 4. Occupied, Single family detached, 1940-1979, electric fuel type, 0-80% AMI   

In [37]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_heating_fuel = ['Natural Gas']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_heating_fuel = ['Electricity']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_heating_fuel = ['Natural Gas']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Detached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_heating_fuel = ['Electricity']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg3_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) &
                      community_results['build_existing_model.heating_fuel'].isin(seg4_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [38]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2679.9
Basic Enclosure                                                     6830.6
Enhanced Enclosure                                                  9962.4
Mininum Efficiency Heat Pump with Existing Heat Backup             22228.2
Mininum Efficiency Heat Pump with Electric Heat Backup             22250.5
Mininum Efficiency Whole Home Electrification                      25026.2
High Efficiency Heat Pump with Electric Heat Backup                26207.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28869.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30729.5
Basic Enclos

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1884.2
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2761.4
Basic Enclosure                                                     7443.2
Enhanced Enclosure                                                  9869.5
Mininum Efficiency Heat Pump with Electric Heat Backup             16978.8
Mininum Efficiency Heat Pump with Existing Heat Backup             16978.8
High Efficiency Heat Pump with Electric Heat Backup                27240.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31274.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32889.1
High Efficiency Whole Home Electrification                         33577.5
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2752.1
Basic Enclosure                                                     6506.5
Enhanced Enclosure                                                  9205.6
Mininum Efficiency Heat Pump with Existing Heat Backup             14552.9
Mininum Efficiency Heat Pump with Electric Heat Backup             14812.6
Mininum Efficiency Whole Home Electrification                      17636.7
High Efficiency Heat Pump with Electric Heat Backup                26358.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31042.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32900.8
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1813.6
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2690.6
Basic Enclosure                                                     6006.3
Enhanced Enclosure                                                  7989.9
Mininum Efficiency Heat Pump with Electric Heat Backup             17261.7
Mininum Efficiency Heat Pump with Existing Heat Backup             17261.7
High Efficiency Heat Pump with Electric Heat Backup                26197.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30362.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31658.1
High Efficie

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [39]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.09699
Enhanced Enclosure                                                 0.07732
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05351
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05078
Basic Enclosure + High Efficiency Whole Home Electrification       0.04831
Heat Pump Water Heater                                             0.04673
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04617
High Efficiency Heat Pump with Electric Heat Backup                0.04345
High Efficiency Whole Home Electrification                         0.04206
Mininum Efficiency Whole Home Electrification                      0.03189
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02842
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02501
Heat Pump Clothes Dryer                                            0.01202
Electric Coo

segment2


upgrade_name
Basic Enclosure                                                    0.13372
Enhanced Enclosure                                                 0.10872
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.06091
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05868
Basic Enclosure + High Efficiency Whole Home Electrification       0.05522
High Efficiency Heat Pump with Electric Heat Backup                0.05385
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.05347
High Efficiency Whole Home Electrification                         0.04944
Mininum Efficiency Heat Pump with Electric Heat Backup             0.04783
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04783
Heat Pump Water Heater                                             0.04466
Mininum Efficiency Whole Home Electrification                      0.01252
Heat Pump Clothes Dryer                                            0.01195
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.10722
Enhanced Enclosure                                                 0.08830
Heat Pump Water Heater                                             0.05143
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04940
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04759
Basic Enclosure + High Efficiency Whole Home Electrification       0.04512
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04372
High Efficiency Heat Pump with Electric Heat Backup                0.04199
Mininum Efficiency Whole Home Electrification                      0.04151
High Efficiency Whole Home Electrification                         0.04096
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03643
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02494
Heat Pump Clothes Dryer                                            0.01318
Electric Coo

segment4


upgrade_name
Basic Enclosure                                                    0.11572
Enhanced Enclosure                                                 0.09727
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.05249
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.05098
High Efficiency Heat Pump with Electric Heat Backup                0.04778
Basic Enclosure + High Efficiency Whole Home Electrification       0.04753
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04636
High Efficiency Whole Home Electrification                         0.04392
Mininum Efficiency Heat Pump with Electric Heat Backup             0.04243
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04243
Heat Pump Water Heater                                             0.04124
Mininum Efficiency Whole Home Electrification                      0.03868
Heat Pump Clothes Dryer                                            0.01368
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [40]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00458
Enhanced Enclosure                                                 0.00366
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00262
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00248
Basic Enclosure + High Efficiency Whole Home Electrification       0.00236
High Efficiency Heat Pump with Electric Heat Backup                0.00230
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00225
Mininum Efficiency Whole Home Electrification                      0.00223
Heat Pump Water Heater                                             0.00220
High Efficiency Whole Home Electrification                         0.00219
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00209
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00181
Electric Cooking                                                   0.00146
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00367
Enhanced Enclosure                                                 0.00298
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00168
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00162
Basic Enclosure + High Efficiency Whole Home Electrification       0.00152
High Efficiency Heat Pump with Electric Heat Backup                0.00148
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00148
Mininum Efficiency Whole Home Electrification                      0.00139
High Efficiency Whole Home Electrification                         0.00137
Electric Cooking                                                   0.00132
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00131
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00131
Heat Pump Water Heater                                             0.00130
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00477
Enhanced Enclosure                                                 0.00393
Mininum Efficiency Whole Home Electrification                      0.00281
Heat Pump Water Heater                                             0.00261
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00261
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00228
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00218
Basic Enclosure + High Efficiency Whole Home Electrification       0.00208
High Efficiency Heat Pump with Electric Heat Backup                0.00206
High Efficiency Whole Home Electrification                         0.00202
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00200
Electric Cooking                                                   0.00159
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00147
Induction Co

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00353
Basic Enclosure                                                    0.00317
Enhanced Enclosure                                                 0.00267
Electric Cooking                                                   0.00148
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00147
Heat Pump Water Heater                                             0.00147
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00143
Basic Enclosure + High Efficiency Whole Home Electrification       0.00134
High Efficiency Heat Pump with Electric Heat Backup                0.00131
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00130
High Efficiency Whole Home Electrification                         0.00124
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00116
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00116
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [41]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.30134
Enhanced Enclosure                                                 0.24255
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.16990
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.16122
Basic Enclosure + High Efficiency Whole Home Electrification       0.15310
Heat Pump Water Heater                                             0.14776
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.14630
High Efficiency Heat Pump with Electric Heat Backup                0.14324
High Efficiency Whole Home Electrification                         0.13798
Mininum Efficiency Whole Home Electrification                      0.12649
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11455
Mininum Efficiency Heat Pump with Existing Heat Backup             0.10161
Electric Cooking                                                   0.08381
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.34302
Enhanced Enclosure                                                 0.28110
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15565
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15030
High Efficiency Heat Pump with Electric Heat Backup                0.14161
Basic Enclosure + High Efficiency Whole Home Electrification       0.14089
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13674
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13413
Mininum Efficiency Heat Pump with Existing Heat Backup             0.13413
High Efficiency Whole Home Electrification                         0.12771
Heat Pump Water Heater                                             0.09194
Electric Cooking                                                   0.08105
Mininum Efficiency Whole Home Electrification                      0.08051
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.31310
Enhanced Enclosure                                                 0.26100
Heat Pump Water Heater                                             0.17243
Mininum Efficiency Whole Home Electrification                      0.15546
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14925
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14362
Mininum Efficiency Heat Pump with Electric Heat Backup             0.13708
Basic Enclosure + High Efficiency Whole Home Electrification       0.13628
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13189
High Efficiency Heat Pump with Electric Heat Backup                0.12907
High Efficiency Whole Home Electrification                         0.12793
Electric Cooking                                                   0.09360
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08721
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.30262
Mininum Efficiency Whole Home Electrification                      0.25558
Enhanced Enclosure                                                 0.25527
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13734
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13350
High Efficiency Heat Pump with Electric Heat Backup                0.12662
Basic Enclosure + High Efficiency Whole Home Electrification       0.12428
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12133
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11896
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11896
High Efficiency Whole Home Electrification                         0.11596
Heat Pump Water Heater                                             0.10872
Electric Cooking                                                   0.09743
Heat Pump Cl

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    24168164.6
Basic Enclosure + High Efficiency Whole Home Electrification       23931069.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23594798.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        23357703.4
High Efficiency Whole Home Electrification                         20844899.8
High Efficiency Heat Pump with Electric Heat Backup                17219305.6
Mininum Efficiency Whole Home Electrification                      12069132.5
Enhanced Enclosure                                                 11647196.8
Basic Enclosure                                                    10018315.5
Mininum Efficiency Heat Pump with Electric Heat Backup              9560695.3
Mininum Efficiency Heat Pump with Existing Heat Backup              8406530.1
Heat Pump Water Heater                                              1893507.5
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    20284435.8
Basic Enclosure + High Efficiency Whole Home Electrification       20026164.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     19984033.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        19725762.4
High Efficiency Whole Home Electrification                         17189065.0
High Efficiency Heat Pump with Electric Heat Backup                15188631.0
Enhanced Enclosure                                                 11110894.2
Basic Enclosure                                                    10305829.6
Mininum Efficiency Heat Pump with Electric Heat Backup              8408329.0
Mininum Efficiency Heat Pump with Existing Heat Backup              8408269.1
Heat Pump Water Heater                                              1196059.8
Heat Pump Clothes Dryer                                              256132.1
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    92945998.1
Basic Enclosure + High Efficiency Whole Home Electrification       91069824.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     90620465.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        88744291.4
High Efficiency Whole Home Electrification                         78041252.5
High Efficiency Heat Pump with Electric Heat Backup                64045519.0
Enhanced Enclosure                                                 47040659.6
Mininum Efficiency Whole Home Electrification                      42363461.4
Basic Enclosure                                                    40374777.2
Mininum Efficiency Heat Pump with Electric Heat Backup             31227785.1
Mininum Efficiency Heat Pump with Existing Heat Backup             21008590.1
Heat Pump Water Heater                                              8145066.7
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    52287276.6
Basic Enclosure + High Efficiency Whole Home Electrification       51652025.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51242037.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50606786.3
High Efficiency Whole Home Electrification                         45676525.3
High Efficiency Heat Pump with Electric Heat Backup                39741905.8
Enhanced Enclosure                                                 24676789.5
Mininum Efficiency Heat Pump with Electric Heat Backup             23256109.1
Mininum Efficiency Heat Pump with Existing Heat Backup             23256033.2
Basic Enclosure                                                    22069000.1
Heat Pump Water Heater                                              3523516.2
Heat Pump Clothes Dryer                                              948020.5
Mininum Efficiency Whole Home Electrification      

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1177261.0
Basic Enclosure + High Efficiency Whole Home Electrification       1170756.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1151729.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1145224.8
High Efficiency Whole Home Electrification                         1086096.1
High Efficiency Heat Pump with Electric Heat Backup                 912898.5
Mininum Efficiency Whole Home Electrification                       845356.4
Mininum Efficiency Heat Pump with Electric Heat Backup              702804.4
Mininum Efficiency Heat Pump with Existing Heat Backup              608256.6
Enhanced Enclosure                                                  551398.4
Basic Enclosure                                                     473429.0
Heat Pump Water Heater                                               89345.1
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    560029.8
Basic Enclosure + High Efficiency Whole Home Electrification       552944.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     550295.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        543210.2
High Efficiency Whole Home Electrification                         475116.6
High Efficiency Heat Pump with Electric Heat Backup                416659.9
Enhanced Enclosure                                                 304798.6
Basic Enclosure                                                    282712.8
Mininum Efficiency Heat Pump with Electric Heat Backup             230660.4
Mininum Efficiency Heat Pump with Existing Heat Backup             230658.8
Heat Pump Water Heater                                              34897.3
Heat Pump Clothes Dryer                                              7026.3
Mininum Efficiency Whole Home Electrification                        4751.0

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4256234.2
Basic Enclosure + High Efficiency Whole Home Electrification       4204765.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4150794.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4099325.5
High Efficiency Whole Home Electrification                         3847362.4
High Efficiency Heat Pump with Electric Heat Backup                3137430.3
Mininum Efficiency Whole Home Electrification                      2868637.0
Mininum Efficiency Heat Pump with Electric Heat Backup             2237221.1
Enhanced Enclosure                                                 2093927.8
Basic Enclosure                                                    1796848.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1235328.3
Heat Pump Water Heater                                              414039.3
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1470897.5
Basic Enclosure + High Efficiency Whole Home Electrification       1453471.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1434356.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1416930.4
High Efficiency Whole Home Electrification                         1289549.5
High Efficiency Heat Pump with Electric Heat Backup                1090023.7
Enhanced Enclosure                                                  676986.0
Mininum Efficiency Heat Pump with Electric Heat Backup              637780.8
Mininum Efficiency Heat Pump with Existing Heat Backup              637778.7
Basic Enclosure                                                     605432.6
Heat Pump Water Heater                                              125327.9
Mininum Efficiency Whole Home Electrification                        52262.7
Heat Pump Clothes Dryer                                        

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    76587755.6
Basic Enclosure + High Efficiency Whole Home Electrification       75845181.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74914264.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        74171690.4
High Efficiency Whole Home Electrification                         68392124.5
High Efficiency Heat Pump with Electric Heat Backup                56766490.2
Mininum Efficiency Whole Home Electrification                      47867485.1
Mininum Efficiency Heat Pump with Electric Heat Backup             38541896.9
Enhanced Enclosure                                                 36538834.1
Mininum Efficiency Heat Pump with Existing Heat Backup             34154251.2
Basic Enclosure                                                    31124606.0
Heat Pump Water Heater                                              5987797.3
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    51877789.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51185662.0
Basic Enclosure + High Efficiency Whole Home Electrification       51099836.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50407708.7
High Efficiency Whole Home Electrification                         44401490.8
High Efficiency Heat Pump with Electric Heat Backup                39943234.0
Enhanced Enclosure                                                 28727164.8
Basic Enclosure                                                    26437403.3
Mininum Efficiency Heat Pump with Electric Heat Backup             23582445.9
Mininum Efficiency Heat Pump with Existing Heat Backup             23582322.9
Heat Pump Water Heater                                              2462022.8
Heat Pump Clothes Dryer                                              503774.1
Mininum Efficiency Whole Home Electrification      

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    280365107.1
Basic Enclosure + High Efficiency Whole Home Electrification       275045104.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     273453930.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        268133927.7
High Efficiency Whole Home Electrification                         243769224.1
High Efficiency Heat Pump with Electric Heat Backup                196874460.1
Mininum Efficiency Whole Home Electrification                      158668716.2
Enhanced Enclosure                                                 139044825.6
Basic Enclosure                                                    117897327.3
Mininum Efficiency Heat Pump with Electric Heat Backup             117507249.1
Mininum Efficiency Heat Pump with Existing Heat Backup              73450763.9
Heat Pump Water Heater                                              27306522.5
Heat Pump Clothes Dryer                

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    136847817.4
Basic Enclosure + High Efficiency Whole Home Electrification       135061772.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     134198375.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        132412330.9
High Efficiency Whole Home Electrification                         120595376.4
High Efficiency Heat Pump with Electric Heat Backup                105326627.7
Mininum Efficiency Heat Pump with Electric Heat Backup              65200101.9
Mininum Efficiency Heat Pump with Existing Heat Backup              65199961.1
Enhanced Enclosure                                                  64762944.2
Basic Enclosure                                                     57713497.0
Heat Pump Water Heater                                               9288693.3
Mininum Efficiency Whole Home Electrification                        3788162.2
Heat Pump Clothes Dryer                

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3926536263.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1598.3
Basic Enclosure + High Efficiency Whole Home Electrification       1582.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1560.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1544.7
High Efficiency Whole Home Electrification                         1378.5
High Efficiency Heat Pump with Electric Heat Backup                1138.7
Mininum Efficiency Whole Home Electrification                       798.2
Enhanced Enclosure                                                  770.3
Basic Enclosure                                                     662.5
Mininum Efficiency Heat Pump with Electric Heat Backup              632.3
Mininum Efficiency Heat Pump with Existing Heat Backup              555.9
Heat Pump Water Heater                                              125.2
Heat Pump Clothes Dryer                                              26.6
Induction Cooking        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1958.9
Basic Enclosure + High Efficiency Whole Home Electrification       1934.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1929.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1905.0
High Efficiency Whole Home Electrification                         1660.0
High Efficiency Heat Pump with Electric Heat Backup                1466.8
Enhanced Enclosure                                                 1073.0
Basic Enclosure                                                     995.3
Mininum Efficiency Heat Pump with Electric Heat Backup              812.0
Mininum Efficiency Heat Pump with Existing Heat Backup              812.0
Heat Pump Water Heater                                              123.3
Heat Pump Clothes Dryer                                              26.4
Mininum Efficiency Whole Home Electrification                        23.6
Induction Cooking        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1606.1
Basic Enclosure + High Efficiency Whole Home Electrification       1573.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1565.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1533.5
High Efficiency Whole Home Electrification                         1348.5
High Efficiency Heat Pump with Electric Heat Backup                1106.7
Enhanced Enclosure                                                  812.8
Mininum Efficiency Whole Home Electrification                       732.0
Basic Enclosure                                                     697.7
Mininum Efficiency Heat Pump with Electric Heat Backup              539.6
Mininum Efficiency Heat Pump with Existing Heat Backup              363.0
Heat Pump Water Heater                                              141.5
Heat Pump Clothes Dryer                                              29.1
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1646.7
Basic Enclosure + High Efficiency Whole Home Electrification       1626.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1613.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1593.8
High Efficiency Whole Home Electrification                         1438.5
High Efficiency Heat Pump with Electric Heat Backup                1251.6
Enhanced Enclosure                                                  777.2
Mininum Efficiency Heat Pump with Electric Heat Backup              732.4
Mininum Efficiency Heat Pump with Existing Heat Backup              732.4
Basic Enclosure                                                     695.0
Heat Pump Water Heater                                              111.0
Mininum Efficiency Whole Home Electrification                        70.1
Heat Pump Clothes Dryer                                              30.2
Induction Cooking        

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3071005957.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3071005957.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3071005957.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    77.9
Basic Enclosure + High Efficiency Whole Home Electrification       77.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     76.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        75.7
High Efficiency Whole Home Electrification                         71.8
High Efficiency Heat Pump with Electric Heat Backup                60.4
Mininum Efficiency Whole Home Electrification                      55.9
Mininum Efficiency Heat Pump with Electric Heat Backup             46.5
Mininum Efficiency Heat Pump with Existing Heat Backup             40.2
Enhanced Enclosure                                                 36.5
Basic Enclosure                                                    31.3
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.1
Basic Enclosure + High Efficiency Whole Home Electrification       53.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.5
High Efficiency Whole Home Electrification                         45.9
High Efficiency Heat Pump with Electric Heat Backup                40.2
Enhanced Enclosure                                                 29.4
Basic Enclosure                                                    27.3
Mininum Efficiency Heat Pump with Electric Heat Backup             22.3
Mininum Efficiency Heat Pump with Existing Heat Backup             22.3
Heat Pump Water Heater                                              3.6
Mininum Efficiency Whole Home Electrification                       2.6
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73.5
Basic Enclosure + High Efficiency Whole Home Electrification       72.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70.8
High Efficiency Whole Home Electrification                         66.5
High Efficiency Heat Pump with Electric Heat Backup                54.2
Mininum Efficiency Whole Home Electrification                      49.6
Mininum Efficiency Heat Pump with Electric Heat Backup             38.7
Enhanced Enclosure                                                 36.2
Basic Enclosure                                                    31.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.3
Heat Pump Water Heater                                              7.2
Electric Cooking                                                    1.6
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    46.3
Basic Enclosure + High Efficiency Whole Home Electrification       45.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     45.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.6
High Efficiency Whole Home Electrification                         40.6
High Efficiency Heat Pump with Electric Heat Backup                34.3
Enhanced Enclosure                                                 21.3
Mininum Efficiency Heat Pump with Electric Heat Backup             20.1
Mininum Efficiency Heat Pump with Existing Heat Backup             20.1
Basic Enclosure                                                    19.1
Mininum Efficiency Whole Home Electrification                       6.4
Heat Pump Water Heater                                              3.9
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/396568384.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/396568384.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/396568384.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    68.8
Basic Enclosure + High Efficiency Whole Home Electrification       68.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        66.6
High Efficiency Whole Home Electrification                         63.3
High Efficiency Heat Pump with Electric Heat Backup                51.3
Mininum Efficiency Whole Home Electrification                      48.4
Mininum Efficiency Heat Pump with Electric Heat Backup             38.5
Mininum Efficiency Heat Pump with Existing Heat Backup             32.7
Enhanced Enclosure                                                 32.2
Basic Enclosure                                                    27.5
Heat Pump Water Heater                                              6.4
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    58.3
Basic Enclosure + High Efficiency Whole Home Electrification       57.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     57.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        56.3
High Efficiency Whole Home Electrification                         49.7
High Efficiency Heat Pump with Electric Heat Backup                42.3
Enhanced Enclosure                                                 30.6
Basic Enclosure                                                    28.1
Mininum Efficiency Heat Pump with Electric Heat Backup             23.2
Mininum Efficiency Heat Pump with Existing Heat Backup             23.2
Heat Pump Water Heater                                              4.7
Mininum Efficiency Whole Home Electrification                       2.8
Electric Cooking                                                    2.0
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.6
Basic Enclosure + High Efficiency Whole Home Electrification       66.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     65.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.0
High Efficiency Whole Home Electrification                         61.2
High Efficiency Heat Pump with Electric Heat Backup                49.1
Mininum Efficiency Whole Home Electrification                      45.2
Mininum Efficiency Heat Pump with Electric Heat Backup             34.6
Enhanced Enclosure                                                 32.8
Basic Enclosure                                                    28.0
Mininum Efficiency Heat Pump with Existing Heat Backup             19.1
Heat Pump Water Heater                                              7.2
Electric Cooking                                                    1.6
Induction Cooking                                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    57.2
Basic Enclosure + High Efficiency Whole Home Electrification       56.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     55.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54.8
High Efficiency Whole Home Electrification                         50.0
High Efficiency Heat Pump with Electric Heat Backup                41.6
Enhanced Enclosure                                                 25.7
Mininum Efficiency Heat Pump with Electric Heat Backup             23.5
Mininum Efficiency Heat Pump with Existing Heat Backup             23.5
Basic Enclosure                                                    22.8
Mininum Efficiency Whole Home Electrification                       7.2
Heat Pump Water Heater                                              5.2
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1358844843.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1358844843.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1358844843.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5064.9
Basic Enclosure + High Efficiency Whole Home Electrification       5015.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4954.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4905.1
High Efficiency Whole Home Electrification                         4522.9
High Efficiency Heat Pump with Electric Heat Backup                3754.1
Mininum Efficiency Whole Home Electrification                      3165.6
Mininum Efficiency Heat Pump with Electric Heat Backup             2548.9
Enhanced Enclosure                                                 2416.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2258.7
Basic Enclosure                                                    2058.3
Heat Pump Water Heater                                              396.0
Electric Cooking                                                     81.9
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5010.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4943.2
Basic Enclosure + High Efficiency Whole Home Electrification       4934.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4868.1
High Efficiency Whole Home Electrification                         4288.0
High Efficiency Heat Pump with Electric Heat Backup                3857.5
Enhanced Enclosure                                                 2774.3
Basic Enclosure                                                    2553.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2277.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2277.4
Heat Pump Water Heater                                              253.9
Mininum Efficiency Whole Home Electrification                       151.7
Electric Cooking                                                     79.2
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4844.6
Basic Enclosure + High Efficiency Whole Home Electrification       4752.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4725.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4633.2
High Efficiency Whole Home Electrification                         4212.2
High Efficiency Heat Pump with Electric Heat Backup                3401.9
Mininum Efficiency Whole Home Electrification                      2741.7
Enhanced Enclosure                                                 2402.6
Basic Enclosure                                                    2037.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2030.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1269.2
Heat Pump Water Heater                                              474.5
Electric Cooking                                                     91.5
Heat Pump Clothes Dryer  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4309.8
Basic Enclosure + High Efficiency Whole Home Electrification       4253.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4226.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4170.1
High Efficiency Whole Home Electrification                         3798.0
High Efficiency Heat Pump with Electric Heat Backup                3317.1
Mininum Efficiency Heat Pump with Electric Heat Backup             2053.4
Mininum Efficiency Heat Pump with Existing Heat Backup             2053.4
Enhanced Enclosure                                                 2039.6
Basic Enclosure                                                    1817.6
Mininum Efficiency Whole Home Electrification                       463.5
Heat Pump Water Heater                                              292.5
Electric Cooking                                                     95.3
Heat Pump Clothes Dryer  

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/624873452.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/624873452.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/624873452.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 3

In [51]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 4

In [52]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [53]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,18.1,7.8
Basic Enclosure + High Efficiency Whole Home Electrification,18.1,8.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,8.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,8.2
High Efficiency Whole Home Electrification,18.1,9.3
High Efficiency Heat Pump with Electric Heat Backup,18.1,10.9
Enhanced Enclosure,18.1,13.0
Mininum Efficiency Whole Home Electrification,18.1,13.1
Basic Enclosure,18.1,13.9


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,18.0,7.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18.0,7.9
Basic Enclosure + High Efficiency Whole Home Electrification,18.0,7.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18.0,8.1
High Efficiency Whole Home Electrification,18.0,9.6
High Efficiency Heat Pump with Electric Heat Backup,18.0,10.7
Enhanced Enclosure,18.0,12.5
Basic Enclosure,18.0,13.0
Mininum Efficiency Heat Pump with Electric Heat Backup,18.0,14.6


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,13.0,5.6
Basic Enclosure + High Efficiency Whole Home Electrification,13.0,5.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,13.0,5.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,13.0,5.9
High Efficiency Whole Home Electrification,13.0,6.8
High Efficiency Heat Pump with Electric Heat Backup,13.0,7.9
Enhanced Enclosure,13.0,9.2
Mininum Efficiency Whole Home Electrification,13.0,9.7
Basic Enclosure,13.0,9.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.0,6.3
Basic Enclosure + High Efficiency Whole Home Electrification,15.0,6.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.0,6.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.0,6.6
High Efficiency Whole Home Electrification,15.0,7.3
High Efficiency Heat Pump with Electric Heat Backup,15.0,8.3
Mininum Efficiency Heat Pump with Existing Heat Backup,15.0,10.9
Mininum Efficiency Heat Pump with Electric Heat Backup,15.0,10.9
Enhanced Enclosure,15.0,11.1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_73042/2330438264.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [54]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')